In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


# ============================
# 1. Pobranie danych MNIST
# ============================

transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

print("MNIST załadowany!")


# ============================
# 2. Wyświetlenie kilku przykładów
# ============================

fig, axes = plt.subplots(2, 5, figsize=(10, 5))
axes = axes.flatten()

for i in range(10):
    img, label = train_dataset[i]
    axes[i].imshow(img.squeeze(), cmap="gray")
    axes[i].set_title(f"Label: {label}")
    axes[i].axis("off")

plt.show()

# ============================
# 3. Podział na loaders
# ============================

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64)

print(len(train_loader), len(test_loader))


In [ ]:
# ============================
# 4. Definicja prostego modelu (sieci neuronowej)
# ============================

class Model(torch.nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.linear1 = torch.nn.Linear(784, 128)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

model = Model()

# ============================
# 5. Trening
# ============================

learning_rate = 1e-3
epochs = 10

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


def train_loop(dataloader, log_every):
    model.train()

    for i, (data, labels) in enumerate(dataloader):
        pred = model(data)
        loss = loss_fn(pred, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % log_every == 0:
            loss = loss.item()
            print(f"Loss: {loss}")


for e in range(epochs):
    print(f"Epoch: {e+1}")
    train_loop(train_loader, 100)


# ============================
# 6. Testowanie
# ============================

correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        preds = model(images)
        predicted = preds.argmax(dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Dokładność na teście: {100 * correct / total:.2f}%")


In [ ]:
# ============================
# 7. Wyświetlenie kilku predykcji
# ============================

fig, axes = plt.subplots(20, 5, figsize=(10, 50))
axes = axes.flatten()

for i in range(100):
    img, label = test_dataset[i]
    axes[i].imshow(img.squeeze(), cmap="gray")
    axes[i].set_title(f"Prediction: {model(img).argmax(dim=1).item()}")
    axes[i].axis("off")

plt.show()